### Imports

In [9]:
import numpy as np
from msvr.model.MSVR import MSVR # https://github.com/Analytics-for-Forecasting/msvr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from es import *

### Get Data

In [10]:
# get curated population from saved file
with open('curated_training_population.pkl', 'rb') as f:
    population = pickle.load(f)

# get random population from saved file
with open('random_training_population.pkl', 'rb') as f:
    population2 = pickle.load(f)

population = population + population2

# get data from population
X = np.array([genome.x for genome in population])
y = np.array([[genome.valid_acc, genome.loss_target_fitness] for genome in population])

# split the data into training and testing sets for validation accuracy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Train and Predict MSVR

In [12]:
# fit the MSVR model
msvr_rbf = MSVR(kernel = 'rbf', C=100)

# fit the model
msvr_rbf.fit(X_train, y_train)

# make predictions
y_pred = msvr_rbf.predict(X_test)

# get the mean absolute error
mae = np.mean(abs(y_pred - y_test))

# print the mean absolute error
print('MSVR (RBF): Mean Absolute Error:', round(mae, 2), '%.')


MSVR (RBF): Mean Absolute Error: 14.04 %.


In [13]:
# fit the MSVR model
msvr_laplace = MSVR(kernel = 'laplacian', C=100)

# fit the model
msvr_laplace.fit(X_train, y_train)

# make predictions
y_pred = msvr_laplace.predict(X_test)

# get the mean absolute error
mae = np.mean(abs(y_pred - y_test))

# print the mean absolute error
print('MSVR (RBF): Mean Absolute Error:', round(mae, 2), '%.')

MSVR (RBF): Mean Absolute Error: 11.11 %.


### Train and Predict with Random Forest Regressor

In [14]:
rfr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rfr.fit(X_train, y_train)

# make predictions
y_pred = rfr.predict(X_test)

# get the mean absolute error
mae = np.mean(abs(y_pred - y_test))

# print the mean absolute error
print('Random Forest Regressor: Mean Absolute Error:', round(mae, 2), '%.')


Random Forest Regressor: Mean Absolute Error: 11.97 %.


### Train and Predict with Voting Regressor

In [15]:
# make predictions with each model
y_pred1 = msvr_rbf.predict(X_test)
y_pred2 = msvr_laplace.predict(X_test)
y_pred3 = rfr.predict(X_test)

# get the average of each prediction
y_pred = (y_pred1 + y_pred2 + y_pred3) / 3

# limit the predictions to the range of 0 to 100
y_pred = np.clip(y_pred, 0, 100)

# get the mean absolute error
mae = np.mean(abs(y_pred - y_test))

# print the mean absolute error
print('Voting Regressor: Mean Absolute Error:', round(mae, 2), '%.')



Voting Regressor: Mean Absolute Error: 10.82 %.
